# Dashboard dos Resultados Finais da Modelagem

## Preparação do Ambiente

In [12]:
import ast
from dash import Dash, dcc, html, Input, Output
import lightgbm as lgb
import logging
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import os
import pandas as pd
import plotly.express as px
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error
import sys
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import warnings

mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)
warnings.filterwarnings("ignore", module="matplotlib.*")
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

## Carregamento dos dados

In [7]:
# Obtém o caminho do diretório do notebook atual (dashboard/)
notebook_dir = os.path.abspath('') 
# Navega para o diretório raiz do projeto (um nível acima de dashboard/)
# Assumindo que o notebook está em `seu_projeto_raiz/dashboard/`
# e as pastas `pipeline/` e `src/` estão em `seu_projeto_raiz/pipeline/src/`
project_root = os.path.dirname(notebook_dir)

# Adiciona o diretório 'pipeline' ao sys.path
# Isso permite importar módulos como `pipeline.src.model_loading`
sys.path.append(os.path.join(project_root, 'pipeline'))

print(f"Diretório raiz do projeto adicionado ao PATH: {project_root}")
print(f"Diretório da pipeline adicionado ao PATH: {os.path.join(project_root, 'pipeline')}")

# Verifique o sys.path (opcional)
# print(sys.path)

Diretório raiz do projeto adicionado ao PATH: C:\Users\Administrador\Desktop\CaseTécnico_Aquarela_TimeSeries
Diretório da pipeline adicionado ao PATH: C:\Users\Administrador\Desktop\CaseTécnico_Aquarela_TimeSeries\pipeline


In [8]:
# Importa a função para carregar artefatos e modelo do MLflow
from src.model_loading import load_all_artifacts_and_params

# Importa a função para calcular métricas
from src.utils import calculate_metrics

In [9]:
# --- Avaliação para modelo LGBM Global ---
def evaluate_lgbm_global_model(model_global, df_test, exog_cols, df_train, categ_cols_global):
    """
    Avalia o modelo global LightGBM no conjunto de teste.
    Inclui MASE usando os dados de treino para escala.
    """


    # Prepara os dados de teste
    df_test_clean = df_test.copy()
    numeric_cols = df_test_clean.select_dtypes(include=['number']).columns
    df_test_clean[numeric_cols] = df_test_clean[numeric_cols].fillna(0)

    y_true = df_test_clean['total_consumption']
    X_test = df_test_clean[exog_cols]

    # Prepara os dados de treino para o cálculo do MASE
    df_train_clean = df_train.copy()
    y_train = df_train_clean['total_consumption']

    # Trata as colunas categóricas
    X_test[categ_cols_global] = X_test[categ_cols_global].astype('category')

    # Faz a previsão
    y_pred = model_global.predict(X_test)

    # Calcula as métricas, incluindo o MASE
    metrics = calculate_metrics(y_true, y_pred, y_train=y_train)

    # Calcula o desvio padrão do consumo real por cliente
    client_std = df_test_clean.groupby('client_id')['total_consumption'].std()
    # Adiciona o desvio padrão do consumo real para todo o conjunto de teste ---
    metrics['y_true_std'] = client_std.mean()

    metrics['client_id'] = 'Global'
    metrics['model'] = 'LGBM Global'
    return pd.DataFrame([metrics])

In [14]:
mlflow_artifacts_path = os.path.join(project_root, 'models', 'artifacts')
MLFLOW_TRACKING_URI = Path(mlflow_artifacts_path).as_uri()
mlflow_experiment_name = "Walk-Forward Validation" # Nome do experimento gerado pela pipeline automatizada, 
#que contém o run principal, o experimento gerado pelo notebook possui o nome 'Treinamento LightGBM Global_v3'
run_name = "Validação Walk-Forward" # Nome do run principal
model_global, df_train, df_val, df_test, exog_cols_global, categ_cols_global = load_all_artifacts_and_params(
    experiment_name=mlflow_experiment_name,
    run_name=run_name, mlflow_tracking_uri=MLFLOW_TRACKING_URI
)
model_global

--- Tentando carregar todos os artefatos e parâmetros do run 'Validação Walk-Forward' ---
✅ Run ID principal encontrado: d46c7c9881f944f9b789ec19b28985b2

⏳ Carregando os parâmetros do run principal...
✅ Parâmetro 'exog_cols' carregado: ['client_id', 'avg_temperature', 'avg_humidity', 'temp_humid_interaction', 'day_counter', 'consumption_lag_1', 'consumption_lag_2', 'consumption_lag_3', 'consumption_lag_4', 'consumption_lag_7', 'consumption_lag_15', 'tipo_feriado', 'day_of_week_name', 'month', 'rolling_mean_3', 'rolling_std_3', 'rolling_mean_7', 'rolling_std_7', 'diff_lag_1']

⏳ Carregando o modelo LGBM do último fold e seus parâmetros...
✅ Parâmetro 'categ_cols' carregado: ['client_id', 'tipo_feriado', 'day_of_week_name', 'month']
✅ Modelo LGBM do último fold (2) carregado com sucesso!

⏳ Baixando e carregando os Dataframes finais...
✅ DataFrames carregados com sucesso!
✅ Diretório temporário './mlflow_downloaded_artifacts_temp' removido.


LGBMRegressor(categorical_feature=['client_id', 'tipo_feriado',
                                   'day_of_week_name', 'month'],
              random_state=42, verbose=-1)

In [15]:
df_train.head()

,client_id,total_consumption,avg_temperature,avg_humidity,temp_humid_interaction,day_counter,consumption_lag_1,consumption_lag_2,consumption_lag_3,consumption_lag_4,consumption_lag_7,consumption_lag_15,rolling_mean_3,rolling_std_3,rolling_mean_7,rolling_std_7,diff_lag_1,day_of_week_name,month,tipo_feriado
date,,,,,,,,,,,,,,,,,,,,
2023-01-01,C0000,18.64,27.090625,59.068750,1600.209355,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,Feriado
2023-01-01,C0003,13.66,27.090625,59.068750,1600.209355,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,Feriado
2023-01-01,C0033,14.37,27.511111,58.372222,1605.884691,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,Feriado
2023-01-01,C0059,17.90,27.511111,58.372222,1605.884691,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,Feriado
2023-01-01,C0096,16.72,25.994444,64.755556,1683.284691,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,Feriado


In [16]:
df_val.head()

,client_id,total_consumption,avg_temperature,avg_humidity,temp_humid_interaction,day_counter,consumption_lag_1,consumption_lag_2,consumption_lag_3,consumption_lag_4,consumption_lag_7,consumption_lag_15,rolling_mean_3,rolling_std_3,rolling_mean_7,rolling_std_7,diff_lag_1,day_of_week_name,month,tipo_feriado
date,,,,,,,,,,,,,,,,,,,,
2023-04-01,C0058,11.73,0.000000,0.000000,0.000000,91,14.45,10.24,12.35,8.11,13.13,7.94,12.346667,2.105002,11.660000,2.049732,-2.72,6,4,Nenhum
2023-04-01,C0011,21.45,26.088235,63.947059,1668.265917,91,14.91,17.65,15.64,20.80,19.60,16.28,16.066667,1.418955,17.388571,2.331505,6.54,6,4,Nenhum
2023-04-01,C0046,13.75,23.796875,60.203125,1432.646240,91,9.95,11.91,10.22,9.90,12.92,9.49,10.693333,1.062277,11.488571,1.582976,3.80,6,4,Nenhum
2023-04-01,C0017,20.32,23.796875,60.203125,1432.646240,91,20.69,15.97,16.29,18.85,18.12,17.87,17.650000,2.637575,18.114286,1.954430,-0.37,6,4,Nenhum
2023-04-01,C0030,19.36,27.261111,58.888889,1605.376543,91,18.97,16.16,18.10,21.22,17.73,20.25,17.743333,1.438553,18.160000,1.633269,0.39,6,4,Nenhum


In [17]:
df_test.head()

,client_id,total_consumption,avg_temperature,avg_humidity,temp_humid_interaction,day_counter,consumption_lag_1,consumption_lag_2,consumption_lag_3,consumption_lag_4,consumption_lag_7,consumption_lag_15,rolling_mean_3,rolling_std_3,rolling_mean_7,rolling_std_7,diff_lag_1,day_of_week_name,month,tipo_feriado
date,,,,,,,,,,,,,,,,,,,,
2023-05-29,C0002,18.70,23.726667,58.740000,1393.704400,149,19.88,18.62,18.66,20.55,16.91,18.29,19.053333,0.716194,19.717143,1.769875,-1.18,1,5,Nenhum
2023-05-29,C0092,10.25,0.000000,0.000000,0.000000,149,10.76,17.09,15.12,12.55,10.38,7.92,14.323333,3.239326,12.967143,2.399026,-0.51,1,5,Nenhum
2023-05-29,C0054,17.67,26.294444,57.844444,1520.987531,149,17.97,16.12,19.90,17.65,20.88,19.34,17.996667,1.890141,18.021429,2.104309,-0.30,1,5,Nenhum
2023-05-29,C0032,19.37,23.726667,58.740000,1393.704400,149,14.92,16.30,22.62,14.37,18.94,17.87,17.946667,4.105622,17.645714,2.798284,4.45,1,5,Nenhum
2023-05-29,C0029,11.11,24.464063,63.200000,1546.128750,149,9.69,10.84,8.68,10.31,8.72,9.47,9.736667,1.080756,10.227143,1.409890,1.42,1,5,Nenhum


In [19]:
categ_cols_global = ['client_id', 'tipo_feriado', 'day_of_week_name', 'month']
lgbm_results = evaluate_lgbm_global_model(model_global, df_test
                                          ,exog_cols_global, df_train,
                                          categ_cols_global )
lgbm_results

C:\Users\Administrador\AppData\Local\Temp\ipykernel_6944\2900159772.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[categ_cols_global] = X_test[categ_cols_global].astype('category')


,mae,rmse,r2,mape,mase,y_true_std,client_id,model
0,0.170678,0.255931,0.995362,1.281104,0.039761,2.256263,Global,LGBM Global


In [20]:
#max(df_train.index) + pd.Timedelta(days=1)
df_test[categ_cols_global] = df_test[categ_cols_global].astype('category')
y_pred = model_global.predict(df_test.drop(columns = ['total_consumption']))
y_pred

array([18.33041778, 10.85102213, 17.54385049, ..., 16.7021437 ,
       15.7470637 , 17.07556284])

## Dashboard

In [21]:
client_ids = df_test['client_id'].unique().sort_values()
num_days_test = 30
dates = df_test.index.unique()
df_test_with_predictions = df_test.loc[:,['client_id', 'total_consumption']]
df_test_with_predictions['prediction'] = y_pred
df_test_with_predictions = df_test_with_predictions.reset_index()

metrics_by_client = {}
for client_id in client_ids:
    df_client = df_test_with_predictions[df_test_with_predictions['client_id'] == client_id]
    df_client = df_client.dropna(subset=['total_consumption', 'prediction'])
    y_true = df_client['total_consumption']
    y_pred = df_client['prediction']
    if not df_client.empty:
        metrics_by_client[client_id] = {
            'MAE': mean_absolute_error(y_true, y_pred),
            'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
            'R²': r2_score(y_true, y_pred),
        }
    else:
        metrics_by_client[client_id] = {
            'MAE': np.nan,
            'RMSE': np.nan,
            'R²': np.nan,
        }


# INICIALIZAÇÃO E LAYOUT DO DASHBOARD ---
app = Dash(__name__)

# Define o layout do aplicativo
app.layout = html.Div(style={'font-family': 'Arial, sans-serif', 'padding': '20px'}, children=[
    html.H1("Dashboard de Previsões de Consumo", style={'textAlign': 'center', 'color': '#333'}),

    html.Div([
        html.H2("Selecione o Cliente:", style={'fontSize': '18px', 'marginBottom': '10px'}),
        dcc.Dropdown(
            id='client-dropdown',
            options=[{'label': client, 'value': client} for client in client_ids],
            value=client_ids[0],  # Valor padrão
            clearable=False,
            style={'width': '50%'}
        )
    ], style={'marginBottom': '20px'}),

    html.Hr(),

    # Contêiner de layout em grid para posicionar as colunas
    # Isso é mais robusto que flex para evitar o bug de crescimento em ambientes como o Colab
    html.Div(style={'display': 'grid', 'gridTemplateColumns': '1fr 2fr', 'gap': '20px'}, children=[
        # Seção de Métricas (coluna esquerda)
        html.Div(style={'minWidth': '300px'}, children=[
            html.H2("Métricas de Desempenho", style={'fontSize': '18px', 'marginBottom': '10px'}),
            html.Div(id='metrics-output', style={'fontSize': '16px', 'color': '#555', 'padding': '10px', 'borderLeft': '3px solid #007bff'}),
        ]),

        # Seção do Gráfico (coluna direita)
        html.Div(children=[
            # Define uma altura fixa para o gráfico para evitar que ele cresça indefinidamente
            dcc.Graph(id='consumption-prediction-graph', style={'height': '500px'})
        ])
    ])
])

# --- 3. CALLBACKS DO DASHBOARD (Lógica de Interatividade) ---

# O callback atualiza os gráficos e métricas com base na seleção do dropdown
@app.callback(
    Output('consumption-prediction-graph', 'figure'),
    Output('metrics-output', 'children'),
    Input('client-dropdown', 'value')
)
def update_dashboard(selected_client_id):
    """
    Função que atualiza o gráfico e as métricas com base no cliente selecionado.
    """
    print(f"💻 Atualizando dashboard para o cliente: {selected_client_id}")

    # Filtra os dados para o cliente selecionado
    df_client = df_test_with_predictions[df_test_with_predictions['client_id'] == selected_client_id]

    # --- Cria o gráfico interativo com Plotly ---
    fig = px.line(df_client, x='date', y=['total_consumption', 'prediction'],
                  title=f"Previsões vs. Real para o Cliente {selected_client_id}",
                  labels={'value': 'Consumo (kWh)', 'variable': 'Tipo de Dado'})

    fig.update_layout(legend_title_text='Legenda')
    fig.update_traces(hovertemplate='Data: %{x}<br>Consumo: %{y:.2f} kWh')

    # --- Prepara a string de métricas para exibição ---
    metrics = metrics_by_client.get(selected_client_id, {})
    metrics_children = [
        html.P(f"MAE: {metrics.get('MAE', 0):.4f}"),
        html.P(f"RMSE: {metrics.get('RMSE', 0):.4f}"),
        html.P(f"R²: {metrics.get('R²', 0):.4f}"),
    ]

    return fig, metrics_children


# --- 4. EXECUÇÃO DO APLICATIVO ---
if __name__ == '__main__':
    print("\n🚀 Iniciando o aplicativo Dash. Clique no link abaixo para visualizá-lo.")
    app.run(debug=True)


🚀 Iniciando o aplicativo Dash. Clique no link abaixo para visualizá-lo.


💻 Atualizando dashboard para o cliente: C0000
